#**ObjectTracking(YOLO-NAS)**

#**Install All the Required Packages**

In [ ]:
!pip install -q super-gradients==3.2.1
!pip install filterpy==1.1.0

#**Import All the Required Libraries**

In [ ]:
import cv2
import torch
from super_gradients.training import models
import numpy as np
import math
from numpy import random
from IPython.display import HTML
from base64 import b64encode
import os

#**Download the Demo Videos**

In [ ]:
!gdown "https://drive.google.com/uc?export=download&id=1Jhb9Kw2HfpOV8NwvG4I3rmJr3vzlGfXT"
!gdown "https://static.vecteezy.com/system/resources/previews/002/018/389/mp4/highway-traffic-merging-free-video.mp4"
!gdown "https://drive.google.com/uc?export=download&id=1IP0bG53LNrczN1dLuf7NmqXl0IuK1EnR"
!gdown "https://drive.google.com/uc?export=download&id=1KcFyOqg4SbjatyjsJWUdC91Rb0lPPlyY"

Downloading...
From: https://drive.google.com/uc?export=download&id=1Jhb9Kw2HfpOV8NwvG4I3rmJr3vzlGfXT
To: /content/OTtest.mp4
100% 1.79M/1.79M [00:00<00:00, 104MB/s]
Downloading...
From: https://static.vecteezy.com/system/resources/previews/002/018/389/mp4/highway-traffic-merging-free-video.mp4
To: /content/highway-traffic-merging-free-video.mp4
4.52kB [00:00, 10.3MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1IP0bG53LNrczN1dLuf7NmqXl0IuK1EnR
To: /content/test4.mp4
100% 635k/635k [00:00<00:00, 54.9MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1KcFyOqg4SbjatyjsJWUdC91Rb0lPPlyY
To: /content/test.mp4
100% 434k/434k [00:00<00:00, 30.7MB/s]


#**Load the Input Video and calculate the Frame Width and Frame Height**

In [ ]:
def get_video_info(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        raise ValueError("Error: Could not open the video file.")

    # Get the frame width and frame height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    return cap, frame_width, frame_height

In [ ]:
video_path = '/content/test.mp4'
cap, frame_width, frame_height = get_video_info(video_path)
print(cap, frame_width, frame_height)


#**Download the Pre-Trained YOLO-NAS Model Weights**

In [ ]:
def load_model(model_name):
    # Check if CUDA (GPU) is available and set the device accordingly
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

    # Load the specified model
    model = models.get(model_name, pretrained_weights="coco").to(device)
    return model


In [ ]:
model_name = 'yolo_nas_s'
model = load_model(model_name)

< cv2.VideoCapture 0x79d3c1b51390> 480 854


[2024-06-01 21:17:37] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-06-01 21:17:38] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


#**1. Object Detection with YOLO-NAS**

# Classes in the COCO Dataset

In [ ]:
def cococlassNames():
  class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella","handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat","baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup","fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli","carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed","diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone","microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors","teddy bear", "hair drier", "toothbrush" ]
  return class_names


In [ ]:
classNames = cococlassNames()

#Prediction

In [ ]:
out=cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 20, (frame_width, frame_height))
count=0
while True:
    ret, frame=cap.read()
    count+=1
    if ret:
        result=list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys=result.prediction.bboxes_xyxy.tolist()
        confidences=result.prediction.confidence
        labels=result.prediction.labels.tolist()

        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox=np.array(bbox_xyxy)
            x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            classname=int(cls)
            class_name=classNames[classname]
            conf=math.ceil((confidence*100))/100
            label=f'{class_name}{conf}'
            print("Frame", count, "", x1, y1,x2, y2, label)

            #Create Bounding Boxes around the Detected Objects
            cv2.rectangle(frame, (x1, y1), (x2, y2), color=(0, 0, 255, 0.6),thickness=2, lineType=cv2.LINE_AA)

            #Create a rectangle above the detected object and add label and confidence score
            t_size=cv2.getTextSize(str(label), 0, fontScale=1/2, thickness=1)[0]
            c2=x1+t_size[0], y1-t_size[1]-3
            cv2.rectangle(frame, (x1, y1), c2, color=(0, 0, 255, 0.6), thickness=-1, lineType=cv2.LINE_AA)
            cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

        out.write(frame)
    else:
        break
out.release()
cap.release()

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
[2024-06-01 21:17:38] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-06-01 21:17:39] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 1  0 405 115 725 car0.91
Frame 1  76 519 179 609 car0.88


[2024-06-01 21:17:40] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 2  77 519 184 610 car0.93
Frame 2  0 404 116 731 car0.92


[2024-06-01 21:17:41] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


...

Frame 90  172 488 248 550 car0.92


[2024-06-01 21:19:40] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 91  172 488 247 550 car0.91


[2024-06-01 21:19:41] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 92  172 488 248 550 car0.91


#Display the Output Demo Video

In [ ]:
!rm '/content/result_compressed.mp4'

In [ ]:
def display_compressed_video(input_path):
    # Compressed video path
    compressed_path = "/content/result_compressed.mp4"

    os.system(f"ffmpeg -i {input_path} -vcodec libx264 {compressed_path}")

    # Show video
    mp4 = open(compressed_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display_html = """
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url

    return HTML(display_html)

In [ ]:
input_video_path = '/content/output.avi'
display_compressed_video(input_video_path)

Frame 93  171 488 247 550 car0.91
